# ABMF Head Office Automation

## Relevant Libraries and variables

In [1]:
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
import time
import os
import glob
import shutil
from datetime import datetime, timedelta
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, StaleElementReferenceException
import re
# from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import sqlite3
import statistics
import calendar
from data_preprocessing_HO import get_csv_files_HO
# from data_preprocessing import clean_data, format_date_list

## Relevant Variables

In [2]:
# Folder where files are downloaded into
base_folder = "C:/Users/Rhoda/Desktop/abmf_data"

# The string used to identify relevant files
prefix = "abmfho"

# Value of 'site' column in dataset
site_value="Head office"

# Value of folder name on local machine
site_folder_name = "Head_office"

holiday_dates = [
    "2023-01-01",
    "2023-05-01",
    "2023-10-01",
    "2023-12-25"
]

start_time_value="08:00:00"
end_time_value="17:00:00"
minus_days_value=["Saturday", "Sunday"]
# holiday_value = format_date_list(holiday_dates)
# past_month_value = (datetime.now() - timedelta(days=30)).strftime('%m')

## Define Website

In [3]:
# Website to automate
website = "https://129.18.155.1/"

# chromedriver location path
# path = "C:/Users/Admin/Documents/chromedriver_win32/chromedriver.exe"

## Set the Folder you want all the file to be downloaded into before sorting


In [4]:
# Set the temporary directory where you want to save the downloaded files
# download_directory = "C:/Users/Admin/Downloads/IoT_data"

# # Change the download directory
# options = webdriver.ChromeOptions()

# prefs = {"download.default_directory": download_directory}
# options.add_experimental_option("prefs", prefs)

## Define Chrome Driver AutoInstaller

In [5]:
# Automatically install the appropriate version of ChromeDriver
chromedriver_autoinstaller.install()

# Initialize the ChromeDriver without specifying the executable_path
# driver = webdriver.Chrome(options=options)
driver = webdriver.Chrome()

In [6]:
# Instruct web driver to navigate to the set website
# driver.get(website)

In [7]:
# Maximum number of retry attempts
max_retries = 5
retry_count = 0

while retry_count < max_retries:
    try:
        driver.get(website)
        # If the page loads successfully, break out of the loop
        break
    except Exception as e:
        print(f"Load Website Retry attempt {retry_count + 1}")
        retry_count += 1
        
        # Print the exception, if it reaches maximum retry attempt
        if retry_count == max_retries:
            print(f"Error: {e}")
            
        # Wait for a few seconds before the next retry
        time.sleep(5)

if retry_count == max_retries:
    print(f"Maximum retries reached. Unable to load {website}")
    driver.quit()

Load Website Retry attempt 1
Load Website Retry attempt 2
Load Website Retry attempt 3
Load Website Retry attempt 4


## Browser Automation

Download the data from the website.

In [16]:
# Navigate to the advanced button element
advanced_button = driver.find_element(by="xpath", value='//div[@class="nav-wrapper"]//button[@id="details-button"]')

MaxRetryError: HTTPConnectionPool(host='localhost', port=62228): Max retries exceeded with url: /session/2792b8e332dd4e5b525ff5a904131e4f/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001DBF54BEAA0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
advanced_button.click()

In [ ]:
# Navigate to the proceed button
time.sleep(8)
proceed_button = driver.find_element(by="xpath", value='//div[@id="details"]/p/a[@id="proceed-link"]')

In [ ]:
# Clicks the proceed_button and retries that action if it fails due to a timeout exception, for a maximum of 5 times.
time.sleep(8)
max_retry_count = 5
retry_count = 0

while retry_count < max_retry_count:
    try:
        # Wait for the element to be clickable (you can adjust the timeout and other conditions as needed)
        proceed_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[@id="details"]/p/a[@id="proceed-link"]')))
        
        # Perform the action on the element
        proceed_button.click()
        
        # If the action succeeds, break out of the loop
        break
    
    # Handle TimeoutException
    except TimeoutException:
        print(f"Attempt {retry_count + 1}: Timed out waiting for the element to be clickable")
        
        if retry_count == 4:
            print("Website is down or internet connection is bad. \nBrowser will shutdown now.")
            driver.quit()
        
        # Increment the retry count
        retry_count += 1
    
    # Handle NoSuchElementException
    except NoSuchElementException:
        print("Bot cannot find relevant element, or very slow internet connection. \nPlease ensure your internet connection is fast and try again. \nBrowser will shutdown now...")
        driver.quit()

In [ ]:
# Select Admin access level
time.sleep(15)
admin = driver.find_element(by="xpath", value='//div[@class="radio"]/label/input[@value="admin"]')

In [ ]:
admin.click()

In [14]:
# Handle potential pop-up alert
# Navigate to the password text box
password_input = driver.find_element(by="xpath", value='//div[@class="form-group"]/input[@type="password"]')

# Input the password
password_input.send_keys('Superadmin')

# Check if the unexpected element pop-up is present
try:
    popup_element = driver.find_element(by="xpath", value='//*[@id="vue-interface-notice"]/div[6]/div/div[2]/button')
    
    # Try to click the element, and handle ElementNotInteractableException
    try:
        popup_element.click()
    except ElementNotInteractableException:
        # Handle the ElementNotInteractableException here, e.g., wait for it to be clickable
        # or perform another action as needed.
        pass

except NoSuchElementException:
    # If the element is not found, do nothing
    pass

In [15]:
# Click the sign in button
sign_in = driver.find_element(by="xpath", value='//div[@class="form-group"]/a[@class="btn btn-lg btn-primary btn-block"]')

In [16]:
sign_in.click()

In [17]:
# Navigate and click the dropdown button
# Maximum number of retry attempts
time.sleep(10)
max_retries = 5

# Define your XPath here
xpath = '//div[@class="collapse navbar-collapse"]/ul/li/a[@class="dropdown-toggle" and contains(text(), "Metering")]'

for retry in range(max_retries):
    try:
        dropdown = driver.find_element(by="xpath", value=xpath)
        # If the element is found, break out of the loop
        break
    except NoSuchElementException:
        if retry < max_retries - 1:
            print(f"Element not found (attempt {retry + 1}/{max_retries}), retrying...")
            time.sleep(1)  # Add a delay before the next retry if needed
        else:
            print("Element not found after maximum retries.")
            # You can raise an exception or handle it accordingly here

In [18]:
dropdown.click()

In [19]:
# Navigate to the datalog option in the dropdown list
time.sleep(10)
datalog = driver.find_element(by="xpath", value='//div[@class="collapse navbar-collapse"]/ul/li/ul/li/a[@href="#!/files/datalog"]')

In [20]:
datalog.click()

In [21]:
# # In the list of files, Find all the file links that match the file name pattern

time.sleep(15)

# Increase the wait time to 30 seconds
wait = WebDriverWait(driver, 30)

try:
    file_links = wait.until(EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, prefix)))
    link_list = []

    # Display text
    for link in file_links:
        # Extract the text from each link element and append it to the link_list
        link_text = link.text
        link_list.append(link_text)

except Exception as e:
    pass
    # print("An error occurred:", str(e))

In [22]:
# link_list

In [23]:
# # Extract dates from link_list
# dates = [re.search(r'\d{4}-\d{2}-\d{2}', link).group() for link in link_list]

# # Filter dates to get the 10th to 19th day of each month
# filtered_dates = [date for date in dates if 10 <= int(date.split('-')[2]) <= 19]

# # Calculate the last month
# last_month = (datetime.now() - timedelta(days=30)).strftime('%Y-%m')

# # Calculate the last date of the current month
# # Get the current date
# current_date = datetime.now()
# # Find the last day of the current month
# last_day_of_current_month = calendar.monthrange(current_date.year, current_date.month)[1]
# # Format the date as year-month-day
# last_date_of_current_month = current_date.replace(day=last_day_of_current_month).strftime('%Y-%m-%d')

# # Calculate the last date of two months ago
# # Get the date two months ago
# two_months_ago = datetime.now() - timedelta(days=60)
# # Find the last day of that month
# last_day_of_two_months_ago = calendar.monthrange(two_months_ago.year, two_months_ago.month)[1]
# # Format the date as year-month-day
# last_date_of_two_months_ago = two_months_ago.replace(day=last_day_of_two_months_ago).strftime('%Y-%m-%d')



# Extract dates from link_list
dates = [re.search(r'\d{4}-\d{2}-\d{2}', link).group() for link in link_list]

# # Filter dates to get the 10th to 19th day of each month
# filtered_dates = [date for date in dates if 10 <= int(date.split('-')[2]) <= 19]

# Calculate the last month
last_month = (datetime.now() - timedelta(days=30)).strftime('%Y-%m')

# Calculate the first date of the current month
# Get the current date
current_date = datetime.now()
# Set the day to the first of the month and format the date as year-month-day
first_date_of_current_month = current_date.replace(day=1).strftime('%Y-%m-%d')

# Calculate the last date of two months ago
# Get the date two months ago
two_months_ago = datetime.now() - timedelta(days=60)
# Find the last day of that month
last_day_of_two_months_ago = calendar.monthrange(two_months_ago.year, two_months_ago.month)[1]
# Format the date as year-month-day
last_date_of_two_months_ago = two_months_ago.replace(day=last_day_of_two_months_ago).strftime('%Y-%m-%d')

In [24]:
base_folder_file_list = []

In [25]:
# Iterate over the files in the specified base folder.
for file in os.listdir(base_folder):
    # Check if the file meets certain conditions:
    
    # Check if the file starts with a specific prefix and has a ".csv" extension.
    if file.startswith(prefix) and file.endswith(".csv"):
        # If the conditions are met, add the file to the list of files in the base folder.
        base_folder_file_list.append(file)

In [26]:
# base_folder_file_list

In [27]:
relevant_month_file_links = []

In [28]:
# Iterate through a list of file links
for file_link in file_links:
    # Iterate through a list containing the current month and the last month
    for date in [last_date_of_two_months_ago, last_month, first_date_of_current_month]:
        # Extract the date from the file link using a regular expression pattern
        file_date = re.search(r'\d{4}-\d{2}-\d{2}', file_link.text).group()
        
        # Check if the current date (current iteration) is present in the file link text
        # Also, verify that the extracted file date is not today's date
        if date in file_link.text and file_date != datetime.now().strftime('%Y-%m-%d'):
            # If the conditions are met, add the file link to a list of relevant file links
            relevant_month_file_links.append(file_link)

In [29]:
# relevant_month_file_links

In [30]:
# len(relevant_month_file_links)

In [31]:
# Download the data for the current month and the past month

# Maximum number of retry attempts
max_retries = 5

In [32]:
# not driver.find_element(by="xpath", value='//*[@id="datalogFiles"]/div[1]/h3')

In [33]:
# Download the data for the current month and the past month

# Wait
time.sleep(5)

# Maximum number of retry attempts
max_retries = 5

retries_1 = 0

expected_file_count = len(relevant_month_file_links)

while retries_1 < max_retries:
    try:
        for file_link in relevant_month_file_links:
            
            retries_2 = 0
            if retries_2 < max_retries:
                try:
                    # Check if the relevant file is in the relevant months, and check if the file has not yet been downloaded
                    if file_link.text not in base_folder_file_list:

                        # Wait for the element to become clickable
                        download_files = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, file_link.text)))

                        # Click the relevant file for downlaod
                        download_files.click()

                        # Add file to the downloaded files list
                        base_folder_file_list.append(file_link.text)

                        # # Exit the loop if the click is successful
                        # break
                        
                except StaleElementReferenceException:
                    
                    # Wait
                    time.sleep(10)
                    
                    # Wait for the element to become clickable
                    download_files = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, file_link.text)))

                    # Click the relevant file for downlaod
                    download_files.click()
                    
                    if retries_2 == 4:
                        print("Automation Bot: Internet unstable or website is broken. \nTry again later...")
                    
                    retries_2 += 1
                    break
                    
                    
            if retries_2 == 5:
                break
                
        if retries_2 == 5:
                break


    except StaleElementReferenceException:
        retries_1 += 1

    except TimeoutException:
        print(f"TimeoutException: Element '{file_link.text}' is not clickable within the specified time.")
        retries_1 += 1
        
    retries_1 += 1
    if retries_1 == 5:
        print("Download process has ended.")
        break

Download process has ended.


In [34]:
# Pause the execution of the script for 60 seconds.
time.sleep(60)

# Initialize a variable to count the number of eligible files.
file_count = 0

# Iterate over the files in the specified base folder.
for file in os.listdir(base_folder):
    # Iterate over the current and last month.
    for date in [last_date_of_two_months_ago, last_month, first_date_of_current_month]:
        # Check if the file meets several conditions:
        # 1. The date is in the file name.
        # 2. The file name starts with the variable prefix.
        # 3. The file has a ".csv" extension.
        # 4. The length of the file name is less than 41 characters.
        if (date in file) and (file.startswith(prefix)) and (file.endswith(".csv")) and (len(file) < 41):
            # If all conditions are met, increment the file count.
            file_count += 1

In [35]:
# Check if the actual file count matches the expected file count.
if expected_file_count == file_count:
    # If they match, print a message indicating that all files have been downloaded.
    print(f"Automation Bot: All files have been downloaded. \nA total of {file_count} file(s)")

# If the expected file count is greater than the actual file count, there are outstanding files.
elif expected_file_count > file_count:
    # Print a message indicating the number of outstanding files to download.
    print(f"Automation Bot: You have {expected_file_count - file_count} outstanding files to download")

# If the expected file count is less than the actual file count, there may be duplicate files.
elif expected_file_count < file_count:
    # Print a message indicating the possibility of downloading duplicate files.
    print("Automation Bot: You may have downloaded duplicate file(s), please check manually.")

Automation Bot: You have 33 outstanding files to download


In [36]:
# Close the WebDriver
driver.quit()

In [37]:
# # Download Data
# # In the list of files, Find all the file links that match the file name pattern "abmfapapa" and download them
# file_links = driver.find_elements(By.PARTIAL_LINK_TEXT, 'abmfapapa')
# for link in file_links:
#     link.click()

## Pre-process the data

In [3]:
# Define a file pattern using the (*) wildcard to match files that start with the prefix and end in .csv
file_pattern = f"{prefix}*.csv"

# Use the glob module to retrieve a list of file paths that match the specified pattern, in the base folder
file_list = glob.glob(os.path.join(base_folder, file_pattern))

In [4]:
raw_files_count = len(file_list)

In [5]:
raw_files_count

31

### Combined all raw data

In [6]:
all_dataframes = []

preprocessed_file_count = 0
raw_file_name_list = []
# Get all files with that start with the prefix and end with .csv in the base folder
for file in os.listdir(base_folder ):
    if file.startswith(prefix) and file.endswith(".csv"):
        file_path = os.path.join(base_folder, file)
        try:
            raw_file_name_list.append(file_path)
            preprocessed_file_count += 1
            df = pd.read_csv(file_path, on_bad_lines="skip")
            all_dataframes.append(df)
        except pd.errors.ParserError as e:
            print(f"Error reading file: {file_path} - {e}")    

In [7]:
# raw_file_name_list

In [8]:
# Concatenate all dataframes into one
combined_df = pd.concat(all_dataframes, ignore_index=True)

# Calculate the last month
last_month = (datetime.now() - timedelta(days=30)).strftime('%Y-%m')
# last_month = '2023-10'

# Convert 'Time' column to datetime type
combined_df['Time'] = pd.to_datetime(combined_df['Time'], errors='coerce')

# combined_df['Time'] = pd.to_datetime(combined_df['Time']).dt.strftime('%Y-%m-%d %H:%M:00')

# Extract year and month from the 'Time' column
combined_df['year_month'] = combined_df['Time'].dt.to_period('M')

# Filter the dataset for the last month
combined_df = combined_df[combined_df['year_month'] == last_month]

# Drop the temporary 'year_month' column if you don't need it
combined_df = combined_df.drop(columns=['year_month'])

# Reset Index
combined_df = combined_df.reset_index(drop=True)

# Convert the "Time" column to datetime objects
combined_df["Time"] = pd.to_datetime(combined_df["Time"], errors='coerce')

# Check if there is any bot created date value
# final_df[final_df["Time"].dt.year == 1970]

# Set the combined df as df
df = combined_df

# Convert the "Time" column to datetime objects
df["Time"] = pd.to_datetime(df["Time"], errors='coerce')

df["Time"] = df["Time"].dt.strftime("%Y-%m-%d %H:%M:00")

df = df.drop_duplicates(['Time'], keep="first")

data = df.copy()

In [9]:
# data.Time

In [10]:
data

,Time,Freq_Hz,Va_V,Vb_V,Vc_V,Vnavg_V,Vab_V,Vbc_V,Vca_V,Vlavg_V,...,DMD_Qs3_kvar,DMD_Ss3_kVA,DI1,DI2,DI3,DI4,DI5,DI6,DI7,DI8
0,2024-10-01 00:00:00,50.623,209.688,208.371,208.575,208.878,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-10-01 00:01:00,50.633,209.847,208.571,208.781,209.067,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-10-01 00:02:00,50.673,210.169,208.869,209.049,209.362,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-10-01 00:03:00,50.713,210.429,209.017,209.212,209.553,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-10-01 00:04:00,50.683,210.213,208.805,208.969,209.329,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32049,2024-10-31 17:32:00,50.905,204.694,203.707,203.927,204.109,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32050,2024-10-31 17:33:00,50.945,204.362,204.086,204.094,204.181,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32051,2024-10-31 17:34:00,50.955,204.373,204.182,204.082,204.212,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32052,2024-10-31 17:35:00,50.915,204.297,203.991,203.995,204.095,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Pre-process the raw data

In [11]:
# Process each file using the relevant functions
# data = pd.read_csv(file_path, on_bad_lines="skip")
# cleaned_data = clean_data(data=data, site_value=site_value, start_time_value=start_time_value, end_time_value=end_time_value,
#                           minus_days_value=minus_days_value, holiday_dates=holiday_dates)

cleaned_data = get_csv_files_HO(data, site_value=site_value, start_time_value=start_time_value, end_time_value=end_time_value, minus_days_value=minus_days_value, holiday_value=holiday_dates)

12443
3604
34565
(34565, 111)


In [12]:
cleaned_data

,Time,Freq_Hz,Va_V,Vb_V,Vc_V,Vnavg_V,Vab_V,Vbc_V,Vca_V,Vlavg_V,...,gen2_litres_model,diesel_prices,grid_tariff,daily_price_of_used,starting_Litres,reported_liter,day,month,year,missing
0,2024-10-01 00:00:00,50.623,209.688,208.371,208.575,208.878,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
1,2024-10-01 00:01:00,50.633,209.847,208.571,208.781,209.067,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
2,2024-10-01 00:02:00,50.673,210.169,208.869,209.049,209.362,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
3,2024-10-01 00:03:00,50.713,210.429,209.017,209.212,209.553,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
4,2024-10-01 00:04:00,50.683,210.213,208.805,208.969,209.329,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34560,2024-10-31 17:32:00,50.905,204.694,203.707,203.927,204.109,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Thursday,OCT,2024,0
34561,2024-10-31 17:33:00,50.945,204.362,204.086,204.094,204.181,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Thursday,OCT,2024,0
34562,2024-10-31 17:34:00,50.955,204.373,204.182,204.082,204.212,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Thursday,OCT,2024,0
34563,2024-10-31 17:35:00,50.915,204.297,203.991,203.995,204.095,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Thursday,OCT,2024,0


In [13]:
# cleaned_data

In [14]:
# Create if not exist folder and save combined raw data for the month
# Convert last_month to a datetime object
# last_month = '2023-10'
# last_month = datetime.strptime(last_month, '%Y-%m')
last_month = (datetime.now() - timedelta(days=30))
# .strftime('%Y-%m')

# Extract year and month from the last_month variable
year = last_month.strftime('%Y')
month_name = last_month.strftime('%B')

# Create the abmf_raw_data folder if it doesn't exist
abmf_raw_data_folder = os.path.join(base_folder, 'abmf_raw_data')
os.makedirs(abmf_raw_data_folder, exist_ok=True)

# Create the year folder inside abmf_raw_data
year_folder = os.path.join(abmf_raw_data_folder, year)
os.makedirs(year_folder, exist_ok=True)

# Create the month folder inside the year folder
month_folder = os.path.join(year_folder, month_name)
os.makedirs(month_folder, exist_ok=True)

# Save the raw DataFrame in the month folder
df.to_csv(os.path.join(month_folder, f'{month_name}_{year}_{site_value}_raw_data.csv'), index=False)

In [15]:
# Create the "Apapa" folder if it doesn't exist
site_folder = os.path.join(base_folder, site_folder_name)

# Create the year folder inside abmf_raw_data
year_folder = os.path.join(site_folder, year)
os.makedirs(year_folder, exist_ok=True)

# Create the month folder inside the year folder
month_folder = os.path.join(year_folder, month_name)
os.makedirs(month_folder, exist_ok=True)

# Save the cleaned_data in the appriopriate directory
cleaned_data.to_csv(os.path.join(month_folder, f'{month_name}_{year}_{site_value}_cleaned_data.csv'), index=False)

In [16]:
if raw_files_count == preprocessed_file_count:
    print(f"All files have been preprocessed. \nAutomation Bot: A total of {preprocessed_file_count} file(s) has been preprocessed.")
else:
    print("You have outstanding data to preprocess")

All files have been preprocessed. 
Automation Bot: A total of 31 file(s) has been preprocessed.


In [17]:
# Delete irrelevant files
for file_path in raw_file_name_list:
    try:
        os.remove(file_path)
        print(f"File deleted: {file_path}")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error deleting file {file_path}: {e}")

File deleted: C:/Users/Rhoda/Desktop/abmf_data\abmfho-2024-09-30T18-58-00-1day.csv
File deleted: C:/Users/Rhoda/Desktop/abmf_data\abmfho-2024-10-01T18-58-00-1day.csv
File deleted: C:/Users/Rhoda/Desktop/abmf_data\abmfho-2024-10-02T18-58-00-1day.csv
File deleted: C:/Users/Rhoda/Desktop/abmf_data\abmfho-2024-10-03T18-58-00-1day.csv
File deleted: C:/Users/Rhoda/Desktop/abmf_data\abmfho-2024-10-04T18-58-00-1day (1).csv
File deleted: C:/Users/Rhoda/Desktop/abmf_data\abmfho-2024-10-04T18-58-00-1day.csv
File deleted: C:/Users/Rhoda/Desktop/abmf_data\abmfho-2024-10-05T18-58-00-1day.csv
File deleted: C:/Users/Rhoda/Desktop/abmf_data\abmfho-2024-10-06T18-58-00-1day.csv
File deleted: C:/Users/Rhoda/Desktop/abmf_data\abmfho-2024-10-07T18-58-00-1day.csv
File deleted: C:/Users/Rhoda/Desktop/abmf_data\abmfho-2024-10-08T18-58-00-1day.csv
File deleted: C:/Users/Rhoda/Desktop/abmf_data\abmfho-2024-10-09T18-58-00-1day.csv
File deleted: C:/Users/Rhoda/Desktop/abmf_data\abmfho-2024-10-10T18-58-00-1day.csv


In [18]:
print("\n\nAutomation Bot: Irrelevant files have been deleted, and cleaned data has been sorted into the relevant folder")



Automation Bot: Irrelevant files have been deleted, and cleaned data has been sorted into the relevant folder


In [19]:
# Clear base download folder

# # base folder
# base_folder = "C:/Users/Admin/Downloads"

## Send data to the database

### Get the pre-processed data

In [20]:
last_month = (datetime.now() - timedelta(days=30)).strftime('%Y-%m')
year = pd.to_datetime(last_month).strftime('%Y')
month_name = pd.to_datetime(last_month).strftime('%B')

# Joining the base folder path with a specific folder named "apapa" using os.path.join().
directory = os.path.join(base_folder, f"{site_folder_name}/{year}/")

# # initialize a variable to store all the dataframes located with the relevant conditions below
# all_dataframes = []

# Walk through the set directory and its subdirectories, and get all files with that start with the prefix and end with .csv
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.startswith(f"{month_name}_{year}_{site_value}") and file.endswith(".csv"):
            print(file)
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path, na_values=["None"])
                # all_dataframes.append(df)
            except pd.errors.ParserError as e:
                print(f"Error reading file: {file_path} - {e}")

# Concatenate all dataframes into one
final_df = df

October_2024_Head office_cleaned_data.csv


In [21]:
final_df["site"]

0        Head office
1        Head office
2        Head office
3        Head office
4        Head office
            ...     
34560    Head office
34561    Head office
34562    Head office
34563    Head office
34564    Head office
Name: site, Length: 34565, dtype: object

In [22]:
final_df["site"] = "Head office"

In [23]:
final_df['site']

0        Head office
1        Head office
2        Head office
3        Head office
4        Head office
            ...     
34560    Head office
34561    Head office
34562    Head office
34563    Head office
34564    Head office
Name: site, Length: 34565, dtype: object

In [24]:
# last_month = (datetime.now() - timedelta(days=30)).strftime('%Y-%m')
# last_month = last_month.strftime('%Y-%m')

In [25]:
# Calculate the last month
# last_month = (datetime.now() - timedelta(days=30)).strftime('%Y-%m')

# Convert 'Time' column to datetime type
final_df['Time'] = pd.to_datetime(final_df['Time'])

# Extract year and month from the 'Time' column
final_df['year_month'] = final_df['Time'].dt.to_period('M')

# Filter the dataset for the last month
final_df = final_df[final_df['year_month'] == last_month]

# Drop the temporary 'year_month' column if you don't need it
final_df = final_df.drop(columns=['year_month'])

# Reset Index
final_df = final_df.reset_index(drop=True)

# Convert the "Time" column to datetime objects
final_df["Time"] = pd.to_datetime(final_df["Time"], errors='coerce')

# Check if there is any bot created date value
# final_df[final_df["Time"].dt.year == 1970]

In [26]:
final_df

,Time,Freq_Hz,Va_V,Vb_V,Vc_V,Vnavg_V,Vab_V,Vbc_V,Vca_V,Vlavg_V,...,gen2_litres_model,diesel_prices,grid_tariff,daily_price_of_used,starting_Litres,reported_liter,day,month,year,missing
0,2024-10-01 00:00:00,50.623,209.688,208.371,208.575,208.878,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
1,2024-10-01 00:01:00,50.633,209.847,208.571,208.781,209.067,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
2,2024-10-01 00:02:00,50.673,210.169,208.869,209.049,209.362,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
3,2024-10-01 00:03:00,50.713,210.429,209.017,209.212,209.553,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
4,2024-10-01 00:04:00,50.683,210.213,208.805,208.969,209.329,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34560,2024-10-31 17:32:00,50.905,204.694,203.707,203.927,204.109,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Thursday,OCT,2024,0
34561,2024-10-31 17:33:00,50.945,204.362,204.086,204.094,204.181,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Thursday,OCT,2024,0
34562,2024-10-31 17:34:00,50.955,204.373,204.182,204.082,204.212,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Thursday,OCT,2024,0
34563,2024-10-31 17:35:00,50.915,204.297,203.991,203.995,204.095,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Thursday,OCT,2024,0


In [27]:
# Set the combined df as df
df = final_df.copy()

### Sort the DataFrame by the 'Time' column
df = df.sort_values(by='Time')

# Reset the index
df = df.reset_index(drop=True)

In [28]:
df

,Time,Freq_Hz,Va_V,Vb_V,Vc_V,Vnavg_V,Vab_V,Vbc_V,Vca_V,Vlavg_V,...,gen2_litres_model,diesel_prices,grid_tariff,daily_price_of_used,starting_Litres,reported_liter,day,month,year,missing
0,2024-10-01 00:00:00,50.623,209.688,208.371,208.575,208.878,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
1,2024-10-01 00:01:00,50.633,209.847,208.571,208.781,209.067,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
2,2024-10-01 00:02:00,50.673,210.169,208.869,209.049,209.362,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
3,2024-10-01 00:03:00,50.713,210.429,209.017,209.212,209.553,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
4,2024-10-01 00:04:00,50.683,210.213,208.805,208.969,209.329,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Tuesday,OCT,2024,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34560,2024-10-31 17:32:00,50.905,204.694,203.707,203.927,204.109,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Thursday,OCT,2024,0
34561,2024-10-31 17:33:00,50.945,204.362,204.086,204.094,204.181,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Thursday,OCT,2024,0
34562,2024-10-31 17:34:00,50.955,204.373,204.182,204.082,204.212,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Thursday,OCT,2024,0
34563,2024-10-31 17:35:00,50.915,204.297,203.991,203.995,204.095,0.0,0.0,0.0,0.0,...,0,1074,69.75,0,0,0,Thursday,OCT,2024,0


### Calculate the 'starting_Litres' and 'daily_price_of_used' in the newly created df columns

In [29]:
# Read the 'start_and_add' data
start_and_add_df = pd.read_csv("startandadd.csv")

In [30]:
# Set the equivalent month names
month_equivalents = {'JAN': 'January', 'FEB': 'February', 'MAR': 'March', 'APR': 'April',
                     'MAY': 'May', 'JUN': 'June', 'JUL': 'July', 'AUG': 'August',
                     'SEP': 'September', 'OCT': 'October', 'NOV': 'November', 'DEC': 'December'}

In [31]:
# Calculate starting litres only once
starting_litres_dict = {}

# Iterate through each row in the DataFrame to find and store starting litres
for index, row in df.iterrows():
    key = (row['site'], row['month'], row['year'])
    
    # Check if starting litres for the current site, month, and year are already calculated
    if key not in starting_litres_dict:
        
        # Find the corresponding row in the start_and_add_df DataFrame
        matching_row = start_and_add_df[
            (start_and_add_df['Branch'] == row['site']) &
            (start_and_add_df['Month'] == month_equivalents.get(row['month'].upper())) &
            (start_and_add_df['Year'] == row['year'])
        ]
        
        # If a matching row is found, store the 'Start_Litres' value in the dictionary
        if not matching_row.empty:
            starting_litres_dict[key] = matching_row['Start_Litres'].values[0]

In [32]:
# Calculate and update 'starting_Litres' in the original DataFrame
for index, row in df.iterrows():
    key = (row['site'], row['month'], row['year'])
    
    # Check if starting litres information is available in the dictionary
    if key in starting_litres_dict:
        # Calculate starting litres by subtracting the stored value from the sum of gen1_litres_model and gen2_litres_model
        if index == 0:
            # For the first row, use the value from the dictionary directly
            df.at[index, 'starting_Litres'] = starting_litres_dict[key]
        else:
            # For subsequent rows, use the calculated value from the previous row
            df.at[index, 'starting_Litres'] = df.at[index - 1, 'starting_Litres'] - (row['gen1_litres_model'] + row['gen2_litres_model'])

In [33]:
# Calculate the value of the column 'daily_price_of_used' in the DataFrame
df['daily_price_of_used'] = ((df['gen1_litres_model'] + df['gen2_litres_model'])) * df['diesel_prices']

In [34]:
# Display the modified DataFrame with selected columns
df[['site', 'month', 'year', 'gen1_litres_model', 'gen2_litres_model', 'starting_Litres', 'diesel_prices', 'daily_price_of_used']]

,site,month,year,gen1_litres_model,gen2_litres_model,starting_Litres,diesel_prices,daily_price_of_used
0,Head office,OCT,2024,0.000000,0,0,1074,0.000000
1,Head office,OCT,2024,0.000000,0,0,1074,0.000000
2,Head office,OCT,2024,0.000000,0,0,1074,0.000000
3,Head office,OCT,2024,0.000000,0,0,1074,0.000000
4,Head office,OCT,2024,0.000000,0,0,1074,0.000000
...,...,...,...,...,...,...,...,...
34560,Head office,OCT,2024,0.109036,0,0,1074,117.104771
34561,Head office,OCT,2024,0.106980,0,0,1074,114.896198
34562,Head office,OCT,2024,0.107430,0,0,1074,115.380035
34563,Head office,OCT,2024,0.106625,0,0,1074,114.514820


### Calculate KPI's

In [35]:
size_per_sqm = 700
gen_capacity_ho = 64
total_op_hours = 559.08

#### Calculate Carbon Footprint

In [36]:
data = {"Aba": {"size_per_sqm": 340}, "Apapa": {"size_per_sqm": 460}, "Badagry": {"size_per_sqm": 380}, "Onitsha 1": {"size_per_sqm": 380}, "Onitsha 2": {"size_per_sqm": 340}, "Dugbe": {"size_per_sqm": 880}, "Head office": {"size_per_sqm": 700}, "Ikeja": {"size_per_sqm": 800}, "Oke arin": {"size_per_sqm": 340}, "Oshodi": {"size_per_sqm": 420}}

In [37]:
df["Carbon_footprint"] = df.apply(lambda row: ((row["grid_kW"] * 0.33) + ((row["gen1_litres_model"] + row["gen2_litres_model"]) * 2.64)) / (data[row["site"]]["size_per_sqm"] * row["system_kWh"]) if (data[row["site"]]["size_per_sqm"] * row["system_kWh"]) != 0 else 0, axis=1)

In [38]:
# list(df.columns)

### Encode the powersource and multipowersource variables

In [39]:
# # One Hot Encode the power source column.
# df = pd.get_dummies(df, columns=['powersource'])

# # One Hot Encode the multipowersource column.
# df = pd.get_dummies(df, columns=['multi_powersource'])

In [40]:
# Save the df as df.csv
df.to_csv('df.csv', index=False)

### Send the combined data to the database

In [41]:
# Establishes a connection to an SQLite database named 'abmf_database.db'
sqliteConnection = sqlite3.connect('abmf2.db')
# sqliteConnection = sqlite3.connect(r'C:\Users\Admin\Desktop\SQL\abmf_database.db')

In [42]:
# SQL query to create an SQLite table named 'combine' with a wide range of columns
sqlite_create_table_query = """
CREATE TABLE IF NOT EXISTS combine (Time datetime, Freq_Hz REAL, Va_V REAL, Vb_V REAL, Vc_V REAL, Vnavg_V REAL, Vab_V REAL, Vbc_V REAL, Vca_V REAL, Vlavg_V REAL,
Ia_A REAL, Ib_A REAL, Ic_A REAL, Iavg_A REAL, PFa REAL, PFb REAL, PFc REAL, PF REAL, I001_A REAL, PF001 REAL, I002_A REAL, PF002 REAL, I003_A REAL, PF003 REAL,
I004_A REAL, PF004 REAL, I005_A REAL, PF005 REAL, I006_A REAL, PF006 REAL, I007_A REAL, PF007 REAL, I008_A REAL, PF008 REAL, I009_A REAL, PF009 REAL, PFs1 REAL,
PFs2 REAL, PFs3 REAL, Pa_kW REAL, Pb_kW REAL, Pc_kW REAL, P_kW REAL, Qa_kvar REAL, Qb_kvar REAL, Qc_kvar REAL, Q_kvar REAL, Sa_kVA REAL, Sb_kVA REAL, Sc_kVA REAL,
S_kVA REAL, P001_kW REAL, Q001_kvar REAL, S001_kVA REAL, P002_kW REAL, Q002_kvar REAL, S002_kVA REAL, P003_kW REAL, Q003_kvar REAL, S003_kVA REAL, P004_kW REAL,
Q004_kvar REAL, S004_kVA REAL, P005_kW REAL, Q005_kvar REAL, S005_kVA REAL, P006_kW REAL, Q006_kvar REAL, S006_kVA REAL, P007_kW REAL, Q007_kvar REAL, S007_kVA REAL,
P008_kW REAL, Q008_kvar REAL, S008_kVA REAL, P009_kW REAL, Q009_kvar REAL, S009_kVA REAL, Ps1_kW REAL, Qs1_kvar REAL, Ss1_kVA REAL, Ps2_kW REAL, Qs2_kvar REAL,
Ss2_kVA REAL, Ps3_kW REAL, Qs3_kvar REAL, Ss3_kVA REAL, site TEXT, gen1_kW REAL, grid_kW REAL, gen2_kW REAL, grid_kWh REAL, gen1_kWh REAL, gen2_kWh REAL, powersource TEXT,
multi_powersource TEXT, system_kW REAL, system_kWh REAL, multi_system_kW REAL, multi_system_kWh REAL, gen1_litres_model REAL,
gen2_litres_model REAL, diesel_prices REAL, grid_tariff REAL, daily_price_of_used REAL, starting_Litres REAL, reported_liter REAL, month REAL,
year TEXT,missing TEXT, Carbon_footprint REAL); """

In [43]:
# Create a cursor object to interact with the SQLite database using the established connection
cursor = sqliteConnection.cursor()

In [44]:
# Display a success message indicating a successful connection to SQLite
print("Successfully Connected to SQLite")

Successfully Connected to SQLite


In [45]:
# Execute the SQL query to create a table in the SQLite database using the cursor
cursor.execute(sqlite_create_table_query)

# Commit the changes to the database to make the table creation permanent
sqliteConnection.commit()

In [46]:
# Read the data to be inserted
# current_data = pd.read_csv("df.csv")
current_data = df

In [47]:
# Construct a SQL query to select all columns from the 'combine' table where the 'site' column matches a specific value
query1 = f"SELECT * FROM combine WHERE site = '{site_value}'"
# query1 = f"SELECT * FROM combine"

In [48]:
# Execute the SQL query 'query1' in the SQLite database and store the result in a pandas DataFrame 'db_data'
db_data = pd.read_sql_query(query1, sqliteConnection)

In [49]:
# db_data

In [51]:
# Concatenate the DataFrames db_data and current_data
df = pd.concat([db_data, current_data], ignore_index=True)

In [52]:
# df

In [53]:
# Remove duplicate rows from DataFrame 'df' based on columns 'Time' and 'site'
# Keeping the last occurrence of each unique combination of 'Time' and 'site'
df.drop_duplicates(subset=['Time','site'], keep='last', inplace=True)
# df.drop_duplicates()

In [54]:
df = df.reset_index(drop=True)

In [55]:
# print(len(list(df.columns)))

In [56]:
df['Time'] = pd.to_datetime(df['Time'])

In [57]:
# Sorts the DataFrame 'df' based on the 'Time' column in ascending order
df.sort_values(by='Time', ascending=False, inplace=True)

In [58]:
# Convert the "Time" column to datetime objects
df["Time"] = pd.to_datetime(df["Time"], errors='coerce')

# Filter rows where the year is 1970
# year_1970_data = df[df["Time"].dt.year == 1970]

In [59]:
df.shape

(739293, 112)

In [60]:
# Drop duplicates
df = df.drop_duplicates(subset=['Time'])

In [61]:
df.shape

(739293, 112)

In [62]:
# df

In [63]:
# Sort and Reset index
df = df.sort_values(by='Time')
df = df.reset_index(drop="true")

In [64]:
# df

In [65]:
# Delete previous table data
query2 = f"DELETE FROM combine WHERE site = '{site_value}'"

In [66]:
# Execute the DELETE statement query
cursor.execute(query2)

In [67]:
# Writing the DataFrame 'df' to an SQLite database table named "combine"
df.to_sql(name="combine", con=sqliteConnection, if_exists='append',index=False)

739293

In [68]:
print(f"Number of rows sent to the database: {df.shape[0]}")

Number of rows sent to the database: 739293


In [69]:
# Commit the changes to the database to make the table creation permanent
sqliteConnection.commit()